In [1]:
import numpy as np
from keras.models import Model
from keras.datasets import mnist
from keras.models import load_model
from sklearn.metrics import label_ranking_average_precision_score

#loading dataset
(x_train,y_train),(x_test,y_test)=mnist.load_data()

#load previously trained model
autoencoder=load_model('autoencoder.h5')
encoder=Model(inputs=autoencoder.input,outputs=autoencoder.get_layer('encoder').output) #to have the output of the encoder only
#array to score the arrays
scores=[]
n_test_samples=1000 #since limited computation power
n_train_samples=[10,50,100,200,300,400,500,750,1000,2000,3000,4000,5000,6000,7000,8000,9000,10000,20000,30000,40000,50000,60000]



/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def compute_average_precision_score(test_codes, test_labels, learned_codes, y_train, n_samples):
    out_distances=[]
    out_labels=[]
    #we store the output distances labels and distances for putting in score
    for i in range(len(test_codes)):
        distances=[]
        
        for code in learned_codes:
            distance=np.linalg.norm(code-test_codes[i])
            distances.append(distance)
   #store the computed distances and labels from train set
        distances=np.array(distances)
    
    #scoring function replaces similar models by 1 and different ones by 0
        labels=np.copy(y_train).astype('float32')
        labels[labels==test_labels[i]]=1
        labels[labels!=test_labels[i]]=0
        labels[labels==-1]=0
        distance_with_labels=np.stack((distances,labels),axis=-1)
        sorted_distance_with_labels=distance_with_labels[distance_with_labels[:,0].argsort()]
   #the distances are between 0 to 28
        sorted_distances=0
        sorted_distances=28-sorted_distance_with_labels[:,0]
        sorted_labels=sorted_distance_with_labels[:,1]

            #we keep only n samples closest elements from the image retrieved
        out_distances.append(sorted_distances[:n_samples])
        out_labels.append(sorted_labels[:n_samples])
            
    out_labels=np.array(out_labels)
    out_labels_file_name = 'computed_data/out_labels_{}'.format(n_samples)
    np.save(out_labels_file_name, out_labels)
    
    out_distances_file_name = 'computed_data/out_distances_{}'.format(n_samples)
    out_distances = np.array(out_distances)
    np.save(out_distances_file_name, out_distances)
    
     # Score the model based on n_samples first images retrieved
    score = label_ranking_average_precision_score(out_labels, out_distances)
    scores.append(score)
    return score

In [17]:
def test_model(n_test_samples,n_train_samples):
    #compute features for training set
    learned_codes=encoder.predict(x_train.reshape(x_train.shape[0],x_train.shape[1],x_train.shape[2],1))
    learned_codes=learned_codes.reshape(learned_codes.shape[0],learned_codes.shape[1]*learned_codes.shape[2]*learned_codes.shape[3])
    
    #compute features for test
    test_set_codes=encoder.predict(x_test.reshape(x_test.shape[0],x_test.shape[1],x_test.shape[2],1))
    test_set_codes=test_set_codes.reshape(test_set_codes.shape[0],test_set_codes.shape[3]*test_set_codes.shape[1]*test_set_codes.shape[2])
    
    #we keep only n samples from the test set as query images
    indexes=np.arange(len(y_test))
    np.random.shuffle(indexes)
    indexes=indexes[:n_test_samples]
    
    #compute the score
    score=compute_average_precision_score(test_set_codes[indexes], y_test[indexes], learned_codes, y_train, n_train_samples)
    
for n_train_sample in n_train_samples:
    test_model(n_test_samples,n_train_samples)

#store the computed scores into a file
np.save('computed_data/scores',np.array(scores))


TypeError: slice indices must be integers or None or have an __index__ method

In [ ]:
def plot_noise_images():
    denoised_images = autoencoder.predict(x_test_noisy.reshape(x_test_noisy.shape[0], x_test_noisy.shape[1], x_test_noisy.shape[2], 1))
    test_img = x_test_noisy[0]
    resized_test_img = cv2.resize(test_img, (280, 280))
    cv2.imshow('input', resized_test_img)
    cv2.waitKey(0)
    output = denoised_images[0]
    resized_output = cv2.resize(output, (280, 280))
    cv2.imshow('output', resized_output)
    cv2.waitKey(0)